# 实现 任务参数 记录

## 数据存储与写入

In [4]:
import yaml
import json
import os
data_dict_ = {}

# 数据读取与写入      
def get_data():
    with open(os.path.join('tmp_data','data.yaml')) as f:
        config = f.read()
        data = yaml.load(config, Loader=yaml.FullLoader)
    return data

# 写入与更新 参数文件
def write_data(renew_data=['key','value']):
    # type:dict
    new_data_dict = get_data()
    if new_data_dict == None:
        new_data_dict = {}
    # 更新数据字典
    print()
    new_data_dict[renew_data[0]] = renew_data[1]
    with open(os.path.join('tmp_data','data.yaml'), 'w') as f:
        data = yaml.dump(new_data_dict, f)



write_data()

### 选择追踪目标

#### 客户端上传视频
- 上传格式 .mp4

In [5]:
@app.route('/get_tar_video', methods=["POST"])
def get_tar_video():
    # 获取上传的视频
    video = request.files.get('video')
    # 文件存储 - 此处注意传输过程中图像选用流式传输，基于二进制形式实现
    with open(os.path.join('tmp_data','tar_video.mp4'), 'wb') as f:
        f.write(data)
    return 0

NameError: name 'app' is not defined

#### 场景目标展示
- 通过运行AI算法对上传的视频中的所有人物进行id分配和裁图存储处理
- 将所有人物的图像保存到了指定文件夹中
- 视野中每个人分配一个文件夹进行图片的保存
- 前后端部分展示各个文件夹中的裁图即可

In [14]:
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/leon_FC')
from Leon_FC import *

def get_tar_crop():
    # 运行指定文件
    path_0 = leon_path_in(r'yolov5_track/track.py')
    path_1 = leon_path_in(r'yolov5_track/yolov5/weights/crowdhuman_yolov5m.pt')
    path_2 = leon_path_in(r'yolov5_track/strong_sort/configs/strong_sort.yaml')
    path_3 = os.path.join('tmp_data','tar_video.mp4')
    com = r'python {}  --yolo-weights {} --config-strongsort {}  --classes 0 --source {}'.format(path_0,path_1,path_2,path_3)
    # print(com)
    os.system(com)
get_tar_crop()

#### 确定追踪目标

In [1]:
import os
from flask import Flask, render_template, Response, make_response,request
app = Flask(__name__)

@app.route('/get_tar', methods=["POST"])
def get_tar():
    get_tar = request.values.get("get_tar")
    write_data(renew_data=['get_tar',get_tar])
    print(request.get_data())
    return 0

# 相关功能函数初始化

## 指定图像的保存
- 是通过类别对各个小个体图像进行选择性的裁图保存
- 对人物的类型分配为两种：['person','tar_person'] （是否为目标人物）
- 通过 数据库 为前后端返回指定数据

In [ ]:
import time
def crop_save(tar_cls,output):
    for key,unit_mes in output['crop'].items():
        cls = unit_mes['cls']
        if cls == tar_cls:
            crop_img = unit_mes['crop_img']
            dir_path = os.path.join('view_data',cls)
            os.makedirs(dir_path, exist_ok=True)
            cv2.imwrite(os.path.join(dir_path,str(time.time()+'.jpg')),crop_img)

## 数据库上传
- 将可视化数据上传至数据库供前后端进行获取与调用
- 此处 主要进行 数据库创建 | 相关表格创建 | 数据插入与更新等操作
- 具体存储使用上传 .zip 文件实现，前后端下载到指定位置后自行解压与处理

### 相关包导入

In [15]:
import pymysql
import os

#### 表创建

In [21]:

# -*- coding: utf-8 -*-
"""
Created on Fri Aug 26 10:22:24 2022

@author: leonzion
"""
db=pymysql.connect(host='localhost',user='LeonZion',password='WSWwsw1234@',database='Leon-PublicPlacesManage-System',port=3306,autocommit=False)
#创建游标对象cursor
cursor=db.cursor()
#创建表的sql
sql='''
    create table Base_Data(
    smoke MediumBlob ,
    tar_person MediumBlob ,
    near_tar MediumBlob
    )
'''
cursor.execute(sql)

OperationalError: (1050, "Table 'base_data' already exists")

### 数据库 - 数据插入

In [43]:
with open('test.zip',  "rb") as zip_file:
    contents = zip_file.read()

In [55]:
#!/usr/bin/python3
 
import pymysql
 
# 打开数据库连接
db = pymysql.connect(host='localhost',user='LeonZion',password='WSWwsw1234@',database='Leon-PublicPlacesManage-System')
 
# 使用cursor()方法获取操作游标 
cursor = db.cursor()

args = tuple([contents,contents,contents])     
sql = "INSERT INTO Base_Data VALUES  (%s,%s,%s);"
cursor.execute(sql,args)  
# 执行sql语句
db.commit()
# 关闭数据库连接
db.close()


## 图像标注

In [1]:
import cv2
# cv2-图像 [x1,y1,x2,y2]-坐标框 贴图文本
def draw_img(img,bbox,text):
    image = img.copy()
    # 左上角
    start_point = (int(bbox[1]),int(bbox[0]))
    # 右下角
    end_point = (int(bbox[3]),int(bbox[2]))
    # draw the rectangle
    cv2.rectangle(image, start_point, end_point, (0, 0, 255), thickness= 3, lineType=cv2.LINE_8) 
    #org: Where you want to put the text
    org = start_point
    # write the text on the input image
    cv2.putText(image, text, org, fontFace = cv2.FONT_HERSHEY_COMPLEX, fontScale = 1.8, color = (0,0,250))
    # display the output
    return image

## 行人检测

### 基本运行代码

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Oct  1 13:22:46 2022

@author: leonzion
"""

from tools.yolo_leon_person import *
from PIL import Image
import datetime
from torchvision.ops import nms, roi_align, roi_pool
import torch.nn.functional as F
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
# 初始化检测器
yo = Yolo_infer_person()
img = Image.open(r'D:\Leon-Coding\Leon_TestData\mans_2.jpg')
yolo_output = yo.infer(img)



C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tools/leon_yolo.pth model, and classes loaded.
Configurations:
----------------------------------------------------------------------
|                     keys |                                   values|
----------------------------------------------------------------------
|               model_path |                      tools/leon_yolo.pth|
|             classes_path |                            tools/cls.txt|
|              input_shape |                               [960, 960]|
|                      phi |                                Leon_nano|
|               confidence |                                      0.5|
|                  nms_iou |                                      0.3|
|          letterbox_image |                                     True|
|                     cuda |                                     True|
----------------------------------------------------------------------


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


### 最终 接口 函数

In [23]:
print(get_dict_dirs(yolo_output))

	- img value_type:PIL.JpegImagePlugin.JpegImageFile
	+ crop
	|   + 0
	|   |   - crop_img value_type:numpy.ndarray
	|   |   - crop_bbox value_type:numpy.ndarray
	|   |   - score value_type:torch.Tensor
	|   |   - fig value_type:torch.Tensor
	|   |   - reid_score value_type:torch.Tensor
	|   + 1
	|   |   - crop_img value_type:numpy.ndarray
	|   |   - crop_bbox value_type:numpy.ndarray
	|   |   - score value_type:torch.Tensor
	|   |   - fig value_type:torch.Tensor
	|   |   - reid_score value_type:torch.Tensor
	|   + 2
	|   |   - crop_img value_type:numpy.ndarray
	|   |   - crop_bbox value_type:numpy.ndarray
	|   |   - score value_type:torch.Tensor
	|   |   - fig value_type:torch.Tensor
	|   |   - reid_score value_type:torch.Tensor
	|   + 3
	|   |   - crop_img value_type:numpy.ndarray
	|   |   - crop_bbox value_type:numpy.ndarray
	|   |   - score value_type:torch.Tensor
	|   |   - fig value_type:torch.Tensor
	|   |   - reid_score value_type:torch.Tensor
	|   + 4
	|   |   - crop_img value_t

## 指定人物追踪

### 逐一获取行人数据

### 重识别的函数设置

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Oct  2 12:58:46 2022

@author: leonzion
"""
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
from torchvision import datasets, models, transforms

import torch
from PIL import Image
import numpy as np
import cv2
import torch.nn.functional as F
import os
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master','model','ft_ResNet50'))
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master'))

from torchvision import datasets, models, transforms

data_transforms = transforms.Compose([
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms_infer = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 基于图片获取特征
def get_fea(path):
    img = data_transforms(Image.open(path))
#     img = torch.tensor(img.transpose(2, 0, 1))
    img = torch.unsqueeze(img, dim = 0)
    img = torch.tensor(img).to(torch.float32)
    net = torch.load('D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net')
    outputs = net(img) 
    # ---- L2-norm Feature ------
    ff = outputs.data.cpu()
    fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
    ff = ff.div(fnorm.expand_as(ff))
    return ff

def get_reidnet(net_path):
    # 'D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net'
    return torch.load(net_path)

def get_fig(net,t):
    # 图像数据转化
    t = torch.unsqueeze(torch.tensor(data_transforms_infer(t)), dim = 0)
    fig = net(t.to(torch.float32))
    # ---- L2-norm Feature ------
    ff = fig.data.cpu()
    fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
    ff = ff.div(fnorm.expand_as(ff))
    return ff

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\torchvision\transforms\transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


### 目标人物特征提取

In [8]:
net = get_reidnet(leon_path_in('models/reid-effi_b0.pth_net')
fig_tar = get_fea(r'D:\Leon-Coding\Leon_TestData\human-22.png')

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [12]:
pwd

'D:\\Leon-Coding\\Leon-PublicPlacesManage-System'

### 群体特征提取与比较

In [9]:
for key,val in yolo_mes.items():
    crop_img = val['crop_img']
    fig = get_fig(net,crop_img)
    yolo_output['crop'][key]['fig'] = fig
    yolo_output['crop'][key]['score'] = F.cosine_similarity(fig,fig_tar,dim=1)

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


### 获取最高相似度

In [10]:
img_orig = np.array(yolo_output['img_orig'][0])
print(type(img_orig))
for key,val in yolo_mes.items():
    crop_img = val['crop_img']
    fig = get_fig(net,crop_img)
    yolo_output['crop'][key]['fig'] = fig
    yolo_output['crop'][key]['score'] = F.cosine_similarity(fig,fig_tar,dim=1)
# 标注最高分
score_list = []
for key,val in yolo_mes.items():
    score_list.append(yolo_output['crop'][key]['score'])

key = str(score_list.index(max(score_list)))
img_orig = draw_img(img_orig,yolo_output['crop'][key]['crop_bbox'],str(yolo_output['crop'][key]['score']))
cv2.imwrite('c.jpg',img_orig)

<class 'numpy.ndarray'>


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


True

### 最终 接口 函数

In [22]:
# 提取函数初始化
net = get_reidnet('D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net')
# 获取 目标人物的特征数组
fig = fig_tar = get_fea(leon_path_in(r'tmp_data\human_2.jpg'))
# 通过接口数据进行特征提取与特征载入
for key,crop_unit in yolo_output['crop'].items():
    fig = get_fig(net,crop_unit['crop_img'])
    yolo_output['crop'][key]['fig'] = fig
    yolo_output['crop'][key]['reid_score'] = F.cosine_similarity(fig,fig_tar,dim=1)

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


### REID 类

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Oct  2 12:58:46 2022

@author: leonzion
"""
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
from torchvision import datasets, models, transforms

import torch
from PIL import Image
import numpy as np
import cv2
import torch.nn.functional as F
import os
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master','model','ft_ResNet50'))
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master'))

from torchvision import datasets, models, transforms

data_transforms = transforms.Compose([
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
data_transforms_infer = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256,128), interpolation=3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 基于图片获取特征
def get_fea(path):
    img = data_transforms(Image.open(path))
#     img = torch.tensor(img.transpose(2, 0, 1))
    img = torch.unsqueeze(img, dim = 0)
    img = torch.tensor(img).to(torch.float32)
    net = torch.load('D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net')
    outputs = net(img) 
    # ---- L2-norm Feature ------
    ff = outputs.data.cpu()
    fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
    ff = ff.div(fnorm.expand_as(ff))
    return ff

def get_reidnet(net_path):
    # 'D:/Leon-Coding/Leon-PublicPlacesManage-System/models/reid-effi_b0.pth_net'
    return torch.load(net_path)

def get_fig(net,t):
    # 图像数据转化
    t = torch.unsqueeze(torch.tensor(data_transforms_infer(t)), dim = 0)
    fig = net(t.to(torch.float32))
    # ---- L2-norm Feature ------
    ff = fig.data.cpu()
    fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
    ff = ff.div(fnorm.expand_as(ff))
    return ff

In [50]:
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
from torchvision import datasets, models, transforms

import torch
from PIL import Image
import numpy as np
import cv2
import torch.nn.functional as F
import os
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master','model','ft_ResNet50'))
leon_path_in(os.path.join('reid','Person_reID_baseline_pytorch-master'))

from torchvision import datasets, models, transforms
class Reid_detect(object):
    def __init__(self):
        # 相关参数初始化
        self.dataset = 'market'
        self.use_id = False
        self.backbone = 'resnet50'
        self.num_cls_dict = { 'market':30, 'duke':23 }
        self.num_ids_dict = { 'market':751, 'duke':702 }
        self.data_transforms_infer = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((256,128), interpolation=3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.data_transforms = transforms.Compose([
            transforms.Resize((256,128), interpolation=3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
        self.net = torch.load(leon_path_in('models/reid-effi_b0.pth_net'))
        self.tar_fig = self.get_fea()
    # 相关函数
    # 对接 yolo_person 获取特征
    def get_figure(self,net,t):
        # 图像数据转化
        t = torch.unsqueeze(torch.tensor(self.data_transforms_infer(t)), dim = 0)
        fig = net(t.to(torch.float32))
        # ---- L2-norm Feature ------
        ff = fig.data.cpu()
        fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
        ff = ff.div(fnorm.expand_as(ff))
        return ff
    def get_fig(self,yolo_output):
        for key,crop_unit in yolo_output['crop'].items():
            fig = self.get_figure(self.net,crop_unit['crop_img'])
            yolo_output['crop'][key]['fig'] = fig
            yolo_output['crop'][key]['reid_score'] = F.cosine_similarity(fig,self.tar_fig,dim=1)
        return yolo_output
    
    # 基于目标任务图片获取特征
    def get_fea(self):
        img = self.data_transforms(Image.open(leon_path_in(r'tmp_data/human_2.jpg')))
        img = torch.unsqueeze(img, dim = 0)
        img = torch.tensor(img).to(torch.float32)
        outputs = self.net(img) 
        # ---- L2-norm Feature ------
        ff = outputs.data.cpu()
        fnorm = torch.norm(ff, p=2, dim=1, keepdim=True)
        ff = ff.div(fnorm.expand_as(ff))
        return ff

### Reid类 示例 接口

In [75]:
reid_det = Reid_detect()
print(reid_det.get_fig(yolo_output))

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'img': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x197565FA508>, 'crop': {'0': {'crop_img': array([[[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       ...,

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128,

## 行人属性识别

### 初始化相关的参数对象

In [51]:
import os
import json
import torch
from PIL import Image

import argparse
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
print(leon_path_in('passer-attribute_get'))
from net import get_model

from torchvision import datasets, models, transforms

# 定义行人属性识别模型的参数对象
class Person_Attribution_Get(object):
    pass
PAG = Person_Attribution_Get()
PAG.dataset = 'market'
PAG.use_id = False
PAG.backbone = 'resnet50'
PAG.num_cls_dict = { 'market':30, 'duke':23 }
PAG.num_ids_dict = { 'market':751, 'duke':702 }
PAG.attri_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size=(288, 144)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# 相关函数
def load_network(network):
    save_path = os.path.join('models','passer_attribute.pth')
    network.load_state_dict(torch.load(save_path))
    print('Resume model from {}'.format(save_path))
    return network
def load_image(path):
    src = Image.open(path)
    src = transforms(src)
    src = src.unsqueeze(dim=0)
    return src
def leon_get_attrmodel():
    num_label, num_id = PAG.num_cls_dict[PAG.dataset], PAG.num_ids_dict[PAG.dataset]
    model_name = '{}_nfc_id'.format(PAG.backbone) if PAG.use_id else '{}_nfc'.format(PAG.backbone)
    model = get_model(model_name, num_label, use_id=PAG.use_id, num_id=num_id)
    model = load_network(model)
    model.eval()
    return model


class predict_decoder(object):

    def __init__(self, dataset):
        with open(leon_path_in(os.path.join(r'passer-attribute_get','doc','label.json')), 'r') as f:
            self.label_list = json.load(f)[dataset]
        with open(leon_path_in(os.path.join(r'passer-attribute_get','doc','attribute.json')), 'r') as f:
            self.attribute_dict = json.load(f)[dataset]
        self.dataset = dataset
        self.num_label = len(self.label_list)

    def decode(self, pred):
        mes_box = []
        pred = pred.squeeze(dim=0)
        for idx in range(self.num_label):
            name, chooce = self.attribute_dict[self.label_list[idx]]
            if chooce[pred[idx]]:
#                 print('{}: {}'.format(name, chooce[pred[idx]]))
                mes_box.append('{}: {}'.format(name, chooce[pred[idx]])) 
        return mes_box

def get_attribute(model,img):
    src = PAG.attri_transforms(img).unsqueeze(dim=0)
    src = model.forward(src)
    pred = torch.gt(src, torch.ones_like(src)/2 )  # threshold=0.5
    Dec = predict_decoder(PAG.dataset)
    mes_box = Dec.decode(pred)
    return [pred,mes_box]

D:\Leon-Coding\Leon-PublicPlacesManage-System\passer-attribute_get


### 实际推理

In [52]:
model = leon_get_attrmodel()
for key,val in yolo_mes.items():
    crop_img = val['crop_img']
    fig = get_fig(net,crop_img)
    yolo_output['crop'][key]['attri'] = get_attribute(model,yolo_output['crop'][key]['crop_img'])[1]
    print(yolo_output['crop'][key]['attri'])
    print('----')

Resume model from models\passer_attribute.pth


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


['age: teenager', 'carrying backpack: no', 'carrying bag: no', 'carrying handbag: no', 'type of lower-body clothing: pants', 'length of lower-body clothing: long lower body clothing', 'sleeve length: short sleeve', 'hair length: long hair', 'wearing hat: no', 'gender: female']
----
['age: teenager', 'carrying backpack: no', 'carrying bag: no', 'carrying handbag: no', 'type of lower-body clothing: pants', 'length of lower-body clothing: short', 'sleeve length: short sleeve', 'hair length: short hair', 'wearing hat: no', 'gender: male']
----
['age: teenager', 'carrying backpack: no', 'carrying bag: no', 'carrying handbag: no', 'type of lower-body clothing: pants', 'length of lower-body clothing: long lower body clothing', 'sleeve length: short sleeve', 'hair length: short hair', 'wearing hat: no', 'gender: female', 'color of upper-body clothing: black', 'color of lower-body clothing: blue']
----
['age: adult', 'carrying backpack: no', 'carrying bag: no', 'carrying handbag: no', 'type of 

### 最终 接口 函数

In [54]:
# 初始化模型
net_person_attribute = leon_get_attrmodel()
# 推理部分
def leon_person_attribute_get(yolo_output):
    for key,val in yolo_output['crop'].items():
        crop_img = val['crop_img']
        fig = get_fig(net,crop_img)
        yolo_output['crop'][key]['attri'] = get_attribute(net_person_attribute,yolo_output['crop'][key]['crop_img'])[1]
    return yolo_output

Resume model from models\passer_attribute.pth


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'crop_img': array([[[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       ...,

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [12

### 行人属性识别 类

In [62]:
import os
import json
import torch
from PIL import Image

import argparse
import sys
if sys.platform == 'win32':
    sys.path.append(r'D:\Leon-Coding\Leon_FC')
else:
    sys.path.append(r'/home/leonzion/Leon_Coding/Leon_FC')
from leon_info import *
from leon_os import *
print(leon_path_in('passer-attribute_get'))
from net import get_model

from torchvision import datasets, models, transforms

class predict_decoder(object):

    def __init__(self, dataset):
        with open(leon_path_in(os.path.join(r'passer-attribute_get','doc','label.json')), 'r') as f:
            self.label_list = json.load(f)[dataset]
        with open(leon_path_in(os.path.join(r'passer-attribute_get','doc','attribute.json')), 'r') as f:
            self.attribute_dict = json.load(f)[dataset]
        self.dataset = dataset
        self.num_label = len(self.label_list)

    def decode(self, pred):
        mes_box = []
        pred = pred.squeeze(dim=0)
        for idx in range(self.num_label):
            name, chooce = self.attribute_dict[self.label_list[idx]]
            if chooce[pred[idx]]:
                mes_box.append('{}: {}'.format(name, chooce[pred[idx]])) 
        return mes_box

class attri_detect(object):
    def __init__(self):
        # 相关参数初始化
        self.dataset = 'market'
        self.use_id = False
        self.backbone = 'resnet50'
        self.num_cls_dict = { 'market':30, 'duke':23 }
        self.num_ids_dict = { 'market':751, 'duke':702 }
        self.attri_transforms = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(size=(288, 144)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
    
        self.model = self.leon_get_attrmodel()
    # 相关函数
    # 模型初始化
    def load_network(self,network):
        save_path = os.path.join('models','passer_attribute.pth')
        network.load_state_dict(torch.load(save_path))
        print('Resume model from {}'.format(save_path))
        return network
    # 图像预处理
    def load_image(self,src):
        src = transforms(src)
        src = src.unsqueeze(dim=0)
        return src
    # 模型获取
    def leon_get_attrmodel(self):
        num_label, num_id = self.num_cls_dict[self.dataset], self.num_ids_dict[self.dataset]
        model_name = '{}_nfc_id'.format(self.backbone) if self.use_id else '{}_nfc'.format(self.backbone)
        model = get_model(model_name, num_label, use_id=self.use_id, num_id=num_id)
        model = self.load_network(model)
        model.eval()
        return model
    # 特征提取函数
    def get_attribute(self,img):
        src = self.attri_transforms(img).unsqueeze(dim=0)
        src =self. model.forward(src)
        pred = torch.gt(src, torch.ones_like(src)/2 )  # threshold=0.5
        Dec = predict_decoder(self.dataset)
        mes_box = Dec.decode(pred)
        return [pred,mes_box]
    # 对接 yolo_person 的特征提取函数
    def leon_person_attribute_get(self,yolo_output):
        for key,val in yolo_output['crop'].items():
            yolo_output['crop'][key]['attri'] = self.get_attribute(yolo_output['crop'][key]['crop_img'])[1]
        return yolo_output

D:\Leon-Coding\Leon-PublicPlacesManage-System\passer-attribute_get


### 类 示例 接口

In [67]:
atr_det = attri_detect()
print(atr_det.leon_person_attribute_get(yolo_output))

Resume model from models\passer_attribute.pth


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'img': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x197565FA508>, 'crop': {'0': {'crop_img': array([[[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       ...,

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128,

## 其他 - 基础检测功能
- 分为三个功能
    - 吸烟检测
    - 摔倒检测
        - 定义摔倒不起检测功能
    - 口罩检测
- 三个功能来自同一个检测模型
    - 需要做一个任务分流函数（基于模型输出进行筛选性操作）

### 模型初始化

In [9]:
def get_yolo_relation():
    pass

### 定义类别映射字典

In [44]:
class_dict = {'smoke':['smoke'],'fall':['fall'],'mask':['face_mask','mask']}
# for key,value in class_dict.items():
#     print(key)
# output: smoke <br> fall <br> mask

### 定义模型检测-输出筛选函数
- 只输出当前任务的检测结果

In [57]:
def choose_opt(mode,output,class_dict):
    # 待删除数据的 键 存储器 - 限于Python字典的性质限制
    mid_list = []
    # 基于类别进行筛选
    for key,unit_mes in output['crop'].items():
        cls = unit_mes['cls']
        # 定义目标类别的变量域
        tar_list = class_dict[mode]
        # 筛选操作
        if cls not in tar_list:
            mid_list.append(key)
    for key in mid_list:
        output['crop'].pop(key, None)
    return output

# test_dict = {'crop':{'smoke':{'cls':'smoke'},'fall':{'cls':'fall'},'mask':{'cls':'mask'}}}
# choose_opt('smoke',test_dict,class_dict)
# output: {'crop': {'smoke': {'cls': 'smoke'}}}

# 图像源 初始化

In [1]:
import cv2
 
class VideoCamera(object):
    def __init__(self):
        # 定义视频数据的图像来源
        self.video = cv2.VideoCapture(0)
    
    def __del__(self):
        self.video.release()
    
    # 设置获取视频的图像数据
    def get_frame(self):
        # 每次运行都通过来源进行数据获取
        success, image = self.video.read()
        # 将图片进行编码，转化为指定的图片格式
        ret, jpeg = cv2.imencode('.jpg', image)
        # 通过字节形式进行流传输
        return jpeg.tobytes()

# 函数功能汇总
- 用于确定选择该项目所选择的AI处理算法
- 需要先读取相关参数再进行对象初始化，然后进行基于AI的图像处理

## 任务参数获取与写入

In [2]:
# 获取所有post请求参数
import os
from flask import Flask, render_template, Response, make_response,request
app = Flask(__name__)

@app.route('/get_prm', methods=["POST"])
def get_prm():
    mes = dict(request.form)
    for key,value in mes.items():
        write_data(renew_data=[key,value])
    return {'opt':True}

# 任务参数解析
```
- 摄像头 camera_id 可选：0，1，2，3


- 追踪对象 tar_person 可选 0,1,2,3,4 ；
    - 适用于行人重识别 <- 相关数据

- 确定任务-类型选择 opt_id 
    - 行人重识别    -0 - 通过yolo_output进行二次检测
    - 行人属性分析  -1
    - 摔倒检测      -2
    - 吸烟检测      -3
    - 口罩检测      -4
    - 可发送列表[1,2,3,4]，搭配 | 结合 --> 使用
```


## 定义 检测对象

In [63]:
class Leon_detect(object):
    def __init__(self,pdm_dict):
        # 摄像头编号
        self.camera_id = pdm_dict['camera_id']
        # 各检测功能是否调用
        self.reid = pdm_dict['reid']
        self.attri = pdm_dict['attri']
        self.smoke_detect = pdm_dict['smoke_detect']
        self.fall_detect = pdm_dict['fall_detect']
        self.mask_detect = pdm_dict['mask_detect']
        # 各模型初始化
        self.person_yolo_detect = Yolo_infer_person()
        self.relation_yolo_detect = get_yolo_relation()
        self.attri_det = attri_detect()
        self.reid_det = Reid_detect()
        # 目标人物的提取特征
        self.tar_fig = self.reid_det.get_fea()

    # 行人检测
    def person_get(self,img):
        yolo_output = self.person_yolo_detect.infer(img)
        return yolo_output
    
    # 行人属性识别
    def person_attri_get(self,yolo_output):
        attri_output = self.attri_det.leon_person_attribute_get(yolo_output)
        return attri_output
        
    # 行人重识别
    def person_reid(self,yolo_output):
        reid_output =  self.reid_det.get_fig(yolo_output)
        return reid_output
    
    # 相关检测
    def relat_det(self,yolo_output):
        pass
    
    # 图像处理函数
    def img_det(self,img):
        mes = self.person_get(img)
        if self.reid:
            mes = self.person_reid(mes)
        if self.attri:
            mes = self.person_attri_get(mes)
        if self.smoke_detect:
            pass
        if self.fall_detect:
            pass
        if self.mask_detect:
            pass
        return mes

In [64]:
pdm_dict = get_data()
leon_det = Leon_detect(pdm_dict)

tools/leon_yolo.pth model, and classes loaded.
Configurations:
----------------------------------------------------------------------
|                     keys |                                   values|
----------------------------------------------------------------------
|               model_path |                      tools/leon_yolo.pth|
|             classes_path |                            tools/cls.txt|
|              input_shape |                               [960, 960]|
|                      phi |                                Leon_nano|
|               confidence |                                      0.5|
|                  nms_iou |                                      0.3|
|          letterbox_image |                                     True|
|                     cuda |                                     True|
----------------------------------------------------------------------


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Resume model from models\passer_attribute.pth


C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\torchvision\transforms\transforms.py:333: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "
C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [65]:
from PIL import Image
img = Image.open(r'D:\Leon-Coding\Leon_TestData\mans_2.jpg')
mes = leon_det.img_det(img)
print(mes)

C:\Users\leonz\.conda\envs\Leon_Deep\lib\site-packages\ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


{'img': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1920x1080 at 0x1752B684A08>, 'crop': {'0': {'crop_img': array([[[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       ...,

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128, 128],
        [128, 128, 128],
        ...,
        [128, 128, 128],
        [128, 128, 128],
        [128, 128, 128]],

       [[128, 128, 128],
        [128, 128,

In [3]:
print(1)

1


# 图像传输

In [2]:
import os
from flask import Flask, render_template, Response, make_response
app = Flask(__name__)
 
# 相机推流 - 创建对象 - 根据图生成二进制传输流 - 在此处对图像进行AI算法的处理
def gen(camera):
    while True:
        # 获取图像
        frame = camera.get_frame()
        # 图像处理
#         img = get_opt(img)
        # 图像传输 - 生成二进制传输流 同时生成包含头部数据和图片数据的传输流
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

# 每次访问，即返回一张处理好的照片 - 注意此处的response是用来进行数据的 反馈操作
@app.route('/video_feed')
def video_feed():
    return Response(gen(VideoCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
 
#当前实时相机画面
@app.route('/cur_camera')
def cur_camera():
    return render_template(r'Flask-StreamData-Trans.html')

# 后端开启

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.7.8:5000
Press CTRL+C to quit
192.168.7.8 - - [09/Oct/2022 23:54:09] "GET /video_feed HTTP/1.1" 200 -
192.168.7.8 - - [09/Oct/2022 23:54:10] "GET /video_feed HTTP/1.1" 200 -
